<a href="https://colab.research.google.com/github/Abyan-1/TubesVisKom/blob/main/tubes_viskom_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# --- 1. EKSTRAK FILE ZIP (UNZIP) ---
local_zip = 'dataset.zip'
target_extract = '/content/dataset_folder'

# Cek apakah file zip ada
if not os.path.exists(local_zip):
    print("❌ ERROR: File 'dataset.zip' tidak ditemukan!")
    print("👉 Pastikan Anda sudah upload file zip dan namanya benar 'dataset.zip'")
    # Hentikan proses jika file tidak ada
else:
    print("📦 Sedang mengekstrak 'dataset.zip'... Tunggu sebentar...")
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall(target_extract)
    zip_ref.close()
    print("✅ Ekstraksi selesai.")

    # --- 2. PERBAIKI PATH FOLDER (OTOMATIS) ---
    # Kita cari di mana folder 'with_mask' dan 'without_mask' berada
    base_dir = None

    # Cek level 1 (langsung di dalam folder ekstraksi)
    isi_level_1 = os.listdir(target_extract)
    if 'with_mask' in isi_level_1 and 'without_mask' in isi_level_1:
        base_dir = target_extract
    else:
        # Cek level 2 (mungkin ada di dalam sub-folder)
        for folder in isi_level_1:
            path_level_2 = os.path.join(target_extract, folder)
            if os.path.isdir(path_level_2):
                isi_level_2 = os.listdir(path_level_2)
                if 'with_mask' in isi_level_2 and 'without_mask' in isi_level_2:
                    base_dir = path_level_2
                    break

    if base_dir is None:
        print("❌ ERROR STRUKTUR FOLDER: Tidak dapat menemukan folder 'with_mask' & 'without_mask'.")
        print(f"Isi folder yang ditemukan: {isi_level_1}")
    else:
        print(f"✅ Path Dataset Ditemukan di: {base_dir}")

        # --- 3. SETUP DATA GENERATOR ---
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=0.2
        )

        train_generator = train_datagen.flow_from_directory(
            base_dir,
            target_size=(150, 150),
            batch_size=32,
            class_mode='binary',
            subset='training'
        )

        validation_generator = train_datagen.flow_from_directory(
            base_dir,
            target_size=(150, 150),
            batch_size=32,
            class_mode='binary',
            subset='validation'
        )

        # --- 4. BUAT MODEL CNN ---
        model = Sequential([
            Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
            MaxPooling2D(2,2),
            Conv2D(64, (3,3), activation='relu'),
            MaxPooling2D(2,2),
            Conv2D(128, (3,3), activation='relu'),
            MaxPooling2D(2,2),
            Flatten(),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # --- 5. MULAI TRAINING ---
        # Pastikan di output tertulis "Found ... images belonging to 2 classes."
        print("\n🚀 MEMULAI TRAINING...")
        history = model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // 32,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // 32,
            epochs=10
        )

        # --- 6. SIMPAN MODEL ---
        model.save('model_masker_fix.h5')
        print("\n🎉 SELESAI! Model disimpan sebagai 'model_masker_fix.h5'. Silakan download.")

📦 Sedang mengekstrak 'dataset.zip'... Tunggu sebentar...
✅ Ekstraksi selesai.
✅ Path Dataset Ditemukan di: /content/dataset_folder/data
Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



🚀 MEMULAI TRAINING...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
115/188 ━━━━━━━━━━━━━━━━━━━━ 17s 246ms/step - accuracy: 0.6835 - loss: 0.6866

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


188/188 ━━━━━━━━━━━━━━━━━━━━ 60s 289ms/step - accuracy: 0.7319 - loss: 0.5848 - val_accuracy: 0.8936 - val_loss: 0.2771
Epoch 2/10
  1/188 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8438 - loss: 0.3995

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - accuracy: 0.8438 - loss: 0.3995 - val_accuracy: 0.8816 - val_loss: 0.2868
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 49s 263ms/step - accuracy: 0.8789 - loss: 0.2817 - val_accuracy: 0.9222 - val_loss: 0.2179
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.8750 - loss: 0.2353 - val_accuracy: 0.9156 - val_loss: 0.2229
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 82s 437ms/step - accuracy: 0.8932 - loss: 0.2536 - val_accuracy: 0.9375 - val_loss: 0.1777
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.9375 - loss: 0.1385 - val_accuracy: 0.9415 - val_loss: 0.1693
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 47s 252ms/step - accuracy: 0.9188 - loss: 0.2118 - val_accuracy: 0.9475 - val_loss: 0.1696
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9375 - loss: 0.1409 - val_accuracy: 0.9468 - val_loss: 0.1678
Epoch 9/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 48s 254ms/step - accuracy: 0.9198 - loss: 0.2036 - val_accurac


🎉 SELESAI! Model disimpan sebagai 'model_masker_fix.h5'. Silakan download.
